<a href="https://colab.research.google.com/github/rustd/MLSamples/blob/master/Regression_with_tensor_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

In [0]:
taxi_far_dataframe = pd.read_csv("taxi-fare-train.csv", sep=",")

In [7]:
taxi_far_dataframe.head()


,vendor_id,rate_code,passenger_count,trip_time_in_secs,trip_distance,payment_type,fare_amount
0,CMT,1,1,1271,3.8,CRD,17.5
1,CMT,1,1,474,1.5,CRD,8.0
2,CMT,1,1,637,1.4,CRD,8.5
3,CMT,1,1,181,0.6,CSH,4.5
4,CMT,1,1,661,1.1,CRD,8.5


In [8]:

taxi_far_dataframe.describe()

,rate_code,passenger_count,trip_time_in_secs,trip_distance,fare_amount
count,100000.0,100000.0,100000.0,100000.0,100000.0
mean,1.0,1.8,682.6,2.8,11.6
std,0.3,1.5,488.0,3.3,9.5
min,0.0,0.0,0.0,0.0,2.5
25%,1.0,1.0,360.0,1.0,6.5
50%,1.0,1.0,554.0,1.7,9.0
75%,1.0,2.0,883.0,3.1,13.0
max,5.0,6.0,9060.0,97.8,225.0


In [9]:
taxi_far_dataframe = taxi_far_dataframe.reindex(
    np.random.permutation(taxi_far_dataframe.index))
taxi_far_dataframe["fare_amount"] /= 50.0
taxi_far_dataframe

,vendor_id,rate_code,passenger_count,trip_time_in_secs,trip_distance,payment_type,fare_amount
98384,VTS,1,1,240,0.6,CRD,0.1
39403,VTS,1,1,420,2.2,CRD,0.2
64706,VTS,1,1,180,1.6,CRD,0.1
3635,CMT,1,1,178,0.5,CRD,0.1
25603,VTS,1,1,240,0.4,CSH,0.1
...,...,...,...,...,...,...,...
6153,CMT,1,2,1390,13.4,CSH,0.8
68418,VTS,1,5,600,1.2,CSH,0.2
64483,VTS,1,1,1500,3.0,CSH,0.3
8123,CMT,1,1,363,1.2,CSH,0.1


In [0]:
# Define the input feature: total_rooms.


my_feature = taxi_far_dataframe[["trip_distance"]]

# Configure a numeric feature column for total_rooms.
feature_columns = [tf.feature_column.numeric_column("trip_distance")]

In [0]:
# Define the label.
targets = taxi_far_dataframe["fare_amount"]

In [0]:
# Use gradient descent as the optimizer for training the model.
my_optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.0000001)
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

# Configure the linear regression model with our feature columns and optimizer.
# Set a learning rate of 0.0000001 for Gradient Descent.
linear_regressor = tf.estimator.LinearRegressor(
    feature_columns=feature_columns,
    optimizer=my_optimizer
)

In [0]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a linear regression model of one feature.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
  
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
      ds = ds.shuffle(buffer_size=10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

In [0]:
_ = linear_regressor.train(
    input_fn = lambda:my_input_fn(my_feature, targets),
    steps=100
)

In [16]:
# Create an input function for predictions.
# Note: Since we're making just one prediction for each example, we don't 
# need to repeat or shuffle the data here.
prediction_input_fn =lambda: my_input_fn(my_feature, targets, num_epochs=1, shuffle=False)

# Call predict() on the linear_regressor to make predictions.
predictions = linear_regressor.predict(input_fn=prediction_input_fn)

# Format predictions as a NumPy array, so we can calculate error metrics.
predictions = np.array([item['predictions'][0] for item in predictions])

# Print Mean Squared Error and Root Mean Squared Error.
mean_squared_error = metrics.mean_squared_error(predictions, targets)
root_mean_squared_error = math.sqrt(mean_squared_error)
print("Mean Squared Error (on training data): %0.3f" % mean_squared_error)
print("Root Mean Squared Error (on training data): %0.3f" % root_mean_squared_error)

Mean Squared Error (on training data): 0.090
Root Mean Squared Error (on training data): 0.300


In [17]:
min_house_value = taxi_far_dataframe["fare_amount"].min()
max_house_value = taxi_far_dataframe["fare_amount"].max()
min_max_difference = max_house_value - min_house_value

print("Min. Median House Value: %0.3f" % min_house_value)
print("Max. Median House Value: %0.3f" % max_house_value)
print("Difference between Min. and Max.: %0.3f" % min_max_difference)
print("Root Mean Squared Error: %0.3f" % root_mean_squared_error)

Min. Median House Value: 0.050
Max. Median House Value: 4.500
Difference between Min. and Max.: 4.450
Root Mean Squared Error: 0.300


In [18]:
calibration_data = pd.DataFrame()
calibration_data["predictions"] = pd.Series(predictions)
calibration_data["targets"] = pd.Series(targets)
calibration_data.describe()

,predictions,targets
count,100000.0,100000.0
mean,0.0,0.2
std,0.0,0.2
min,0.0,0.1
25%,0.0,0.1
50%,0.0,0.2
75%,0.0,0.3
max,0.0,4.5


In [0]:
sample = taxi_far_dataframe.sample(n=300)